In [36]:
# https://scikit-learn.org/stable/auto_examples/linear_model/plot_tweedie_regression_insurance_claims.html
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (
    FunctionTransformer,
    KBinsDiscretizer,
    OneHotEncoder,
    StandardScaler,
)

import pandas as pd
import numpy as np

In [37]:
df = pd.read_csv('data/insurance_claims.csv')
df.head()

,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount
0,1.0,0.10,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82,0.0
1,1.0,0.77,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82,0.0
2,1.0,0.75,B,6.0,2.0,52.0,50.0,B12,Diesel,54.0,R22,0.0
3,1.0,0.09,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72,0.0
4,1.0,0.84,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72,0.0


In [38]:
# 异常值处理
df['ClaimAmount'] = df['ClaimAmount'].clip(upper=200000)
df['Exposure'] = df['Exposure'].clip(upper=1)
df['ClaimNb'] = df['ClaimNb'].clip(upper=4)

# 如果索赔金额为0，那么我们不将其视为索赔。损失函数
# 严重性模型所使用的#需要严格要求的正索赔金额。就这样了
# 频率和严重程度更加一致。
# ClaimNb：索赔的数量
df.loc[(df['ClaimAmount'] == 0) & (df['ClaimNb'] >= 1), 'ClaimNb'] = 0

log_scale_transformer = make_pipeline(
    FunctionTransformer(func=np.log), StandardScaler()
)

column_trans = ColumnTransformer(
    [
        (
            'binned_numeric',
            KBinsDiscretizer(n_bins=10, random_state=0),
            ['VehAge', 'DrivAge'],
        ),
        (
            'onehot_categorical',
            OneHotEncoder(),
            ['VehBrand', 'VehPower', 'VehGas', 'Region', 'Area'],
        ),
        ('passthrough_numeric', 'passthrough', ['BonusMalus']),
        ('log_scaled_numeric', log_scale_transformer, ['Density']),
    ],
    remainder='drop',
)

x_sparse = column_trans.fit_transform(df)

# 保险公司对建模纯保费很感兴趣
# 每个投保人每单位风险暴露的预期总索赔金额
# 在他们的投资组合中：
df['PurePremium'] = df['ClaimAmount'] / df['Exposure']

# 这可以通过一个两步建模来间接近似：的乘积
# 频率乘以每次索赔的平均索赔金额：
df['Frequency'] = df['ClaimNb'] / df['Exposure']
df['AvgClaimAmount'] = df['ClaimAmount'] / np.fmax(df['ClaimNb'], 1)

with pd.option_context('display.max_columns', 15):
    print(df[df.ClaimAmount > 0].head())

print(df.shape)
df.head()

     ClaimNb  Exposure Area  VehPower  VehAge  DrivAge  BonusMalus VehBrand  \
66       1.0      0.75    F       7.0     1.0     61.0        50.0      B12   
93       1.0      0.14    B      12.0     5.0     50.0        60.0      B12   
199      1.0      0.14    E       4.0     0.0     36.0        85.0      B12   
205      2.0      0.62    F      10.0     0.0     51.0       100.0      B12   
223      1.0      0.31    A       5.0     0.0     45.0        50.0      B12   

      VehGas  Density Region  ClaimAmount   PurePremium  Frequency  \
66   Regular  27000.0    R11       303.00    404.000000   1.333333   
93    Diesel     56.0    R25      1981.84  14156.000000   7.142857   
199  Regular   4792.0    R11      1456.55  10403.928571   7.142857   
205  Regular  27000.0    R11     10834.00  17474.193548   3.225806   
223  Regular     12.0    R73      3986.67  12860.225806   3.225806   

     AvgClaimAmount  
66           303.00  
93          1981.84  
199         1456.55  
205         5417

,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount,PurePremium,Frequency,AvgClaimAmount
0,0.0,0.10,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82,0.0,0.0,0.0,0.0
1,0.0,0.77,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82,0.0,0.0,0.0,0.0
2,0.0,0.75,B,6.0,2.0,52.0,50.0,B12,Diesel,54.0,R22,0.0,0.0,0.0,0.0
3,0.0,0.09,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72,0.0,0.0,0.0,0.0
4,0.0,0.84,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72,0.0,0.0,0.0,0.0


In [39]:
# 将 CSR 矩阵转换为稠密矩阵
x_dense = x_sparse.toarray()

# 将稠密矩阵保存为 CSV 文件
df_x = pd.DataFrame(x_dense)

# 定义稠密矩阵的表头
dense_header = ['X{}'.format(idx) for idx in range(df_x.shape[1])]

# 重命名稠密矩阵的表头
df_x.columns = dense_header

print(df_x.shape)
df_x.head()

(678013, 75)


,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,50.0,0.600055
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,50.0,0.600055
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,50.0,-1.065404
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,50.0,-0.882694
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,50.0,-0.882694


In [40]:
df['idx'] = df.index
df_x['idx'] = df.index

In [41]:
df.head()

,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount,PurePremium,Frequency,AvgClaimAmount,idx
0,0.0,0.10,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82,0.0,0.0,0.0,0.0,0
1,0.0,0.77,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82,0.0,0.0,0.0,0.0,1
2,0.0,0.75,B,6.0,2.0,52.0,50.0,B12,Diesel,54.0,R22,0.0,0.0,0.0,0.0,2
3,0.0,0.09,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72,0.0,0.0,0.0,0.0,3
4,0.0,0.84,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72,0.0,0.0,0.0,0.0,4


In [42]:
df_x.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X66,X67,X68,X69,X70,X71,X72,X73,X74,idx
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,50.0,0.600055,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,50.0,0.600055,1
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,50.0,-1.065404,2
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,50.0,-0.882694,3
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,50.0,-0.882694,4


In [43]:
df_res = pd.merge(df, df_x, on='idx', how='left')

print(df_res.shape)
df_res.head()

(678013, 91)


,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,...,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74
0,0.0,0.10,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,50.0,0.600055
1,0.0,0.77,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,50.0,0.600055
2,0.0,0.75,B,6.0,2.0,52.0,50.0,B12,Diesel,54.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,50.0,-1.065404
3,0.0,0.09,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,50.0,-0.882694
4,0.0,0.84,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,50.0,-0.882694


In [44]:
# 过滤出有效样本
df_res = df_res[df_res['ClaimAmount'] > 0]
print('df shape: {}, df_avaliable shape: {}.'.format(df.shape, df_res.shape))

df shape: (678013, 16), df_avaliable shape: (24944, 91).


In [45]:
# 创建新的索引列
raw_id_col = range(1, len(df_res) + 1)

# 将新索引列插入到数据框最前面
df_res.insert(0, 'raw_id', raw_id_col)
df_res.insert(1, 'example_id', raw_id_col)
df_res.insert(2, 'event_date', '20240901')

df_res.head()

,raw_id,example_id,event_date,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,...,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74
66,1,1,20240901,1.0,0.75,F,7.0,1.0,61.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,50.0,2.257113
93,2,2,20240901,1.0,0.14,B,12.0,5.0,50.0,60.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,60.0,-1.045961
199,3,3,20240901,1.0,0.14,E,4.0,0.0,36.0,85.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,85.0,1.332797
205,4,4,20240901,2.0,0.62,F,10.0,0.0,51.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,100.0,2.257113
223,5,5,20240901,1.0,0.31,A,5.0,0.0,45.0,50.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,50.0,-1.869529


In [46]:
# 导出，特征文件
df_res.to_csv('data/insurance_claims_label_feature_fed.csv', index=False)

print(df_res.shape)

(24944, 94)


In [35]:
aaa = ['X{}'.format(idx) for idx in range(75)]

print(aaa)

['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74']
